# News Classification

## Load Dataset

In [1]:
import numpy as np
import pandas as pd
dataframe = pd.read_csv('news_data.csv')
dataframe.info()

dataframe.groupby('category').describe()

/home/sourish/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sourish/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200853 entries, 0 to 200852
Data columns (total 2 columns):
text        200853 non-null object
category    200853 non-null object
dtypes: object(2)
memory usage: 3.1+ MB


text         \
                count unique   
category                       
ARTS             1509   1509   
ARTS & CULTURE   1339   1338   
BLACK VOICES     4528   4525   
BUSINESS         5937   5935   
COLLEGE          1144   1144   
COMEDY           5175   5165   
CRIME            3405   3401   
CULTURE & ARTS   1030   1029   
DIVORCE          3426   3423   
EDUCATION        1004   1004   
ENTERTAINMENT   16058  16054   
ENVIRONMENT      1323   1322   
FIFTY            1401   1401   
FOOD & DRINK     6226   6226   
GOOD NEWS        1398   1398   
GREEN            2622   2617   
HEALTHY LIVING   6694   6689   
HOME & LIVING    4195   4163   
IMPACT           3459   3458   
LATINO VOICES    1129   1129   
MEDIA            2815   2813   
MONEY            1707   1707   
PARENTING        8677   8649   
PARENTS          3955   3897   
POLITICS        32739  32722   
QUEER VOICES     6314   6311   
RELIGION         2556   2549   
SCIENCE          2178   2178   
SPORTS           4884   4884   
STYLE            2254   2247   
STYLE & BEAUTY   9649   9516   
TASTE            2096   2096   
TECH             2082   2033   
TRAVEL           9887   9884   
WEDDINGS         3651   3651   
WEIRD NEWS       2670   2670   
WELLNESS        17827  17822   
WOMEN            3490   3405   
WORLD NEWS       2177   2176   
WORLDPOST        6243   6243   

                                                                        
                                                              top freq  
category                                                                
ARTS            This Winter, You Can Get Tattooed At A Famous ...    1  
ARTS & CULTURE  This New York Activist Wants To Replace A Stat...    2  
BLACK VOICES    'Black Love' Wedding Announcements (PHOTOS). "...    4  
BUSINESS        How Can Businesses Build Trust?. Putting in pl...    2  
COLLEGE         Signs to Good Words. American Sign Language st...    1  
COMEDY                Saturday's Morning Email: Funnies Edition.     5  
CRIME                              Ferguson Police Officer Shot.     2  
CULTURE & ARTS  Sanjay Theodore on the ImageBlog. All photogra...    2  
DIVORCE         What I Wish People Learned About Marriage and ...    2  
EDUCATION       Progressive Labels for Regressive Practices: H...    1  
ENTERTAINMENT              Rihanna Soaks Up The Sun In A Bikini.     2  
ENVIRONMENT     10 Cities That Could Run Out Of Water - Weathe...    2  
FIFTY           Why I No Longer Pursue Happiness. When I was a...    1  
FOOD & DRINK    Winter Wonderful Hot Chocolate Cheesecake. Thi...    1  
GOOD NEWS             Gratitude For My Friend's Pragmatic Heart.     1  
GREEN           Take A Look At The Best Animal Photos Of The W...    3  
HEALTHY LIVING            Days of Inspiration for the New Year!.     5  
HOME & LIVING   Weekly Roundup of eBay Vintage Home Finds (PHO...    6  
IMPACT          Veterans for Hire: How Companies Can Help Them...    2  
LATINO VOICES   A Show All About Fighters. This week on Latino...    1  
MEDIA           Condé Nast Names New CEO, Bob Sauerberg, To St...    2  
MONEY           The 5 Greatest Money Lessons I Learned From My...    1  
PARENTING       Best Parenting Tweets: What Moms And Dads Said...   26  
PARENTS         Funniest Parenting Tweets: What Moms And Dads ...   23  
POLITICS        The 'Alt-Lite' White House Reporter Who Got Ar...    2  
QUEER VOICES    A Look Back At Some Of The Week's Biggest LGBT...    2  
RELIGION        This Week In Pictures: Faith In Practice Aroun...    8  
SCIENCE         The SESAME Laboratory: Celebrating the Power o...    1  
SPORTS                         The Time Is Now For Bradley Beal.     1  
STYLE             Weekly Roundup of eBay Vintage Clothing Finds.     4  
STYLE & BEAUTY  Fashionably Late Style Quiz: Test Your Knowled...   17  
TASTE           How To Actually Get A Bartender's Attention. P...    1  
TECH            Watch The Top 9 YouTube Videos Of The Week. If...   46  
TRAVEL      

### Separate Train variables and Result

In [2]:
X = dataframe['text']
y = dataframe['category']

### Number of instances of each Label type

In [3]:
print(y.value_counts())

POLITICS          32739
WELLNESS          17827
ENTERTAINMENT     16058
TRAVEL             9887
STYLE & BEAUTY     9649
PARENTING          8677
HEALTHY LIVING     6694
QUEER VOICES       6314
WORLDPOST          6243
FOOD & DRINK       6226
BUSINESS           5937
COMEDY             5175
SPORTS             4884
BLACK VOICES       4528
HOME & LIVING      4195
PARENTS            3955
WEDDINGS           3651
WOMEN              3490
IMPACT             3459
DIVORCE            3426
CRIME              3405
MEDIA              2815
WEIRD NEWS         2670
GREEN              2622
RELIGION           2556
STYLE              2254
SCIENCE            2178
WORLD NEWS         2177
TASTE              2096
TECH               2082
MONEY              1707
ARTS               1509
FIFTY              1401
GOOD NEWS          1398
ARTS & CULTURE     1339
ENVIRONMENT        1323
COLLEGE            1144
LATINO VOICES      1129
CULTURE & ARTS     1030
EDUCATION          1004
Name: category, dtype: int64


## Preprocessing

### Remove Duplicate Labels

In [4]:
from preprocessing import removeDuplicateLabels
y = removeDuplicateLabels(y)

### Encode Labels

In [5]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

### Clean Text data

In [6]:
from preprocessing import cleanText
X_clean = []
for i in range(len(X)):
    X_clean.append(cleanText(X[i]))
X_clean

['mass shooting texas last week tv left husband killed child another day america',
 'smith join diplo nicky jam world cup official song course song',
 'hugh grant marries first time age actor longtime girlfriend anna eberstein tied knot civil ceremony',
 'jim carrey blast castrato adam schiff democrat new artwork actor give dems as kicking fighting hard enough donald trump',
 'julianna margulies us donald trump poop bag pick dog dietland actress said using bag really cathartic therapeutic moment',
 'morgan freeman devastated sexual harassment claim could undermine legacy right equate horrific incident sexual assault misplaced compliment humor said statement',
 'donald trump lovin new mcdonald jingle tonight show bit catchy right',
 'watch amazon prime new week great mini series joining week',
 'mike myers reveals like fourth austin power film myer kid may pushing new power film anyone',
 'watch hulu new week getting recent academy award winning movie',
 'justin timberlake visit texas s

In [7]:
X = X_clean

### Feature Extraction using TF-IDF

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect  = TfidfVectorizer(analyzer='word', token_pattern=r'[a-zA-Z]{2,}', strip_accents='unicode',
                               stop_words='english', ngram_range=(1,2), max_features=50000)
X = vect.fit_transform(X)

## Classification

### Split Train and Test set

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape)
print(X_test.shape)

(160682, 50000)
(40171, 50000)


### Define Classification model

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

classifier = LinearSVC()

### Training

In [11]:
classifier.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

### Testing

In [12]:
pred = classifier.predict(X_test)

### Classification Report

In [13]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pred, target_names=labelencoder.classes_))

                precision    recall  f1-score   support

ARTS & CULTURE       0.55      0.47      0.51       772
  BLACK VOICES       0.58      0.43      0.49       932
      BUSINESS       0.55      0.47      0.51      1185
         CRIME       0.58      0.58      0.58       685
       DIVORCE       0.83      0.73      0.77       707
     EDUCATION       0.49      0.45      0.47       420
 ENTERTAINMENT       0.69      0.74      0.71      4320
   ENVIRONMENT       0.55      0.50      0.53       799
         FIFTY       0.42      0.17      0.24       270
  FOOD & DRINK       0.75      0.79      0.77      1673
     GOOD NEWS       0.48      0.25      0.33       292
 HOME & LIVING       0.69      0.81      0.74      5797
        IMPACT       0.43      0.29      0.35       713
 LATINO VOICES       0.70      0.32      0.44       251
         MEDIA       0.62      0.49      0.55       563
         MONEY       0.48      0.33      0.39       326
     PARENTING       0.66      0.73      0.69  